In [11]:
import random
from bisect import bisect_left


#Generate N elements for each layer and select M out of N
class Element: #define a element
 def __init__(self, eID, layer, monitored):
    self.id = eID
    self.layer = layer
    self.monitored = monitored

 def infoOut(self):
    print("Element info: id = "
          + str(self.id)
          + ", layer = " 
          + str(self.layer) 
          + ", monitored = " 
          + str(self.monitored) 
         ) 

class EvtQueue:
 def __init__(self):
    self.evtID = 0 #initial number of events
    self.queue = [] #empty queue
                
 def getNextEvt(self): #obtain the head of line fault in the evt queue
    if (len(self.queue) > 0):
        currFault = self.queue[0]
        self.queue.remove(currFault) #delete the currFault 
        return currFault 
    else:
        return None
    
 def hasNextEvt(self): #check if the queue is not empty
    if (len(self.queue) > 0):
        return True
    else:
        return False
        
 def getEvtNum(self):
    return self.evtID
    
 def infoOut(self):
    for f in self.queue:
        f.infoOut()
    
 def insertEvt(self, newFault):
    def insert(seq, keys, item, keyfunc=lambda v: v):
        """Insert an item into a sorted list using a separate corresponding
       sorted keys list and a keyfunc() to extract the key from each item.
       Based on insert() method in SortedCollection recipe:
       http://code.activestate.com/recipes/577197-sortedcollection/
        """
        k = keyfunc(item)  # Get key.
        i = bisect_left(keys, k)  # Determine where to insert item.
        keys.insert(i, k)  # Insert key of item to keys list.
        seq.insert(i, item)  # Insert the item itself in the corresponding place.
    keys = [fault.evtTime for fault in self.queue]
    insert(self.queue, keys, newFault, keyfunc=lambda x: x.evtTime) #insert on the order of evtTime to the queue
    self.evtID = self.evtID + 1
    
#Generate a seqence of faults sorted in an increasing order of evtTime
class Fault: #define a fault
 def __init__(self, fID, initlayer, layer, initTime, evtTime, assocElement, impactVector):
    self.id = fID
    self.initLayer = initlayer
    self.layer = layer
    self.initTime = initTime #the inital time when the fault is generated for calculating delay
    self.evtTime = evtTime #the time when the fault is re-inserted in the event queue
    self.assocElement = assocElement #within N elements
    self.impactVector = impactVector #impact vector of this fault
    
 def infoOut(self):
    print("Fault info: id = "
          + str(self.id)
          + ", initLayer = " 
          + str(self.initLayer) 
          + ", layer = " 
          + str(self.layer) 
          + ", initTime = " 
          + str(self.initTime) 
          + ", evtTime = " 
          + str(self.evtTime) 
          + ", assocElementID = " 
          + str(self.assocElement.id)
          + ", monitored = " 
          + str(self.assocElement.monitored)
          + ", impactVector = " 
          + str(self.impactVector)          
         )  
    
#This class manages all elements init, update in the system
class Elements:
    def __init__(self, N, M_N):
        self.M_N = M_N
        self.M = []
        self.N = N
        for i in range(len(N)):
            self.M.append(self.N[i]*self.M_N)
    
        self.elements = []
        elementID = 0
        for i in range(len(N)): #for each layer i
            j = 0
            elements_layer = []
            while j < self.N[i]: #generate Ni elements
                elementID = elementID + 1
                elements_layer.append(Element(elementID, i, False))
                j = j + 1
            count = 0
            assert(self.M[i] <= self.N[i]), "Total elements Ni is too small to reach Mi!"
            while count < self.M[i]: #select Mi elements out of Ni 
                randomElement = random.randrange(N[i])
                #print(randomElement)
                if elements_layer[randomElement].monitored == False:
                    elements_layer[randomElement].monitored = True
                    count = count + 1
            self.elements.append(elements_layer)
    
    #Update elements based on a given M_N, randomly select M out of N, but no change of the array 
    def update_elements(self, M_N):
        self.M_N = M_N
        #print(self.M)
        #print (self.M_N)
        #print(self.N)
        for i in range(len(self.M)):
            self.M[i] = self.N[i]*self.M_N
        #print(self.M)
        
        for i in range (I): #for each layer i
            j = 0
            elements_layer = self.elements[i]
            while j < self.N[i]: #generate Ni elements
                #elementID = elementID + 1
                elements_layer[j].monitored = False #reset all to False
                j = j + 1
            
            count = 0
            assert(self.M[i] <= self.N[i]), "Total elements Ni is too small to reach Mi!"
            while count < self.M[i]: #select Mi elements out of Ni 
                randomElement = random.randrange(self.N[i])
                #print(randomElement)
                if elements_layer[randomElement].monitored == False:
                    elements_layer[randomElement].monitored = True
                    count = count + 1
        
    def infoOut(self):
        for i in range (I): #for each layer i
            j = 0
            elements_layer = self.elements[i]
            strOut = ""
            while j < self.N[i]: #generate Ni elements
                #elementID = elementID + 1
                strOut = strOut + str(elements_layer[j].monitored) + " " 
                j = j + 1
            strOut = strOut + "\n"
            print(strOut)
        
def generate_impactVector(i, Ni): #generate an impact vector with a random impact value at element j at layer i, where element j is randomly selected among the Ni number of elements of layer i
    vector = []
    #if (i in range (0, Ni): #assign all zeros to the ImpactVector
    #    vector[i] = 0
    vector.append(i) #vector[0] = i, the layer i
    j = random.randint(0, Ni-1)
    vector.append(j) #vector[1] = j, the element j
    vector.append(random.random()) #vector[2] = the impact value
    return vector
                   

""" 
Initialization
"""
LASTTIME = 1000; #simulaton running time
I = 5 # number of layers 0 ~ (I-1)
LAMDAS = [0.01, 0.08, 0.3] #the fault generation rates for each experiment for each layer
M_NS = [0.1, 0.5, 0.8]
SEED = 1
random.seed(SEED)
THRESHOLD_IMPACTVECTOR = 0.5
DELTA = 5 #the time for a fault to be generated by impact factor 

#Input loop variables
option = 4
D_0 = 10 # the largest delay
D_1 = 8 # a large delay
D_2 = 5
D_3 = 3

failureRatio = LAMDAS[2]
M_N = M_NS[1]
N = [10, 10, 10, 10, 10] #number of elements for layer i


#Counter for monitored/detected/diagnosed faults: 
C_M = 0 #monitored 
C_D = 0 #detected
C_R = 0 #diagnosed
faultsTotal = 0 #faults != events in queue, one fault may trigger multiple events at different layers for option 4
delayTotal = 0
impactedFaultsTotal = 0

accuracyFaultDetection = 0.9
accuracyRCA = 0.8
accuracyCorrelation = 0.7

#Threshold
# C_R/faultsTotal = M/N * accuracyFaultDetection * accuracyRCA 
THRESHOLD_UP = 0.9 #M_N*0.8
THRESHOLD_DOWN = 0.4 #M_N*0.3 #double check here
M_N_Change_Rate = 0.3
THRESHOLD_FAULTS = 1000
timesThresholdFaults = 1

#initialize elements monitored and not-monitored at each layer
elements = Elements(N, M_N)
elements.infoOut()
queue = EvtQueue()
# for each layer, insert the initial fault with evtTime = 0 in the queue


for i in range(I): 
    evtTime = 0
    elements_layer = elements.elements[i] #find corresponding elements_layer 
    elementIndex = random.randint(0, len(elements_layer)-1) #randomly (uniformaly) assign fault to Ni elements a..t layer i
    if i > 0: #Topdown: if i is not the lowest layer, the impact vector will impact to i-1 layer 
        newFault = Fault(faultsTotal, i, i, evtTime, evtTime, 
                         elements_layer[elementIndex], generate_impactVector(i-1, N[i-1]))   
    else: #for the lowest layer, the impact vector is None
        newFault = Fault(faultsTotal, i, i, evtTime, evtTime, 
                         elements_layer[elementIndex], None)  
    queue.insertEvt(newFault)
    faultsTotal = faultsTotal + 1
#queue.infoOut()


#start to process a Head of Line fault in the event queue
while (queue.hasNextEvt()):
    currFault = queue.getNextEvt()
    assert(currFault != None), "Empty queue!"
    #currFault.infoOut()

   
    """ 
    Fault generation
    """
    #New fault generation  
    #insert the next new fault based on the info of the currFault
    #this new fault generaton is to follow the poission process of new fault generation
    #it is not related to the fault generated due to impact vector
    #only generate a new fault for the current fault with its initTime == evtTime
    if currFault.evtTime == currFault.initTime:
        nextEvtTime = currFault.evtTime + random.expovariate(failureRatio)
        if nextEvtTime <= LASTTIME:
            elementIndex = random.randint(0, len(elements.elements[currFault.layer])-1) #generate a new fault with the same layer as the current fault
            if i > 0: #Topdown: if i is not the lowest layer, the impact vector will impact to i-1 layer 
                newFault = Fault(faultsTotal, currFault.layer, currFault.layer, nextEvtTime, nextEvtTime, 
                         elements.elements[currFault.layer][elementIndex], generate_impactVector(currFault.layer-1, N[currFault.layer-1]))   
            else: #for the lowest layer, the impact vector is None
                newFault = Fault(faultsTotal, currFault.layer, currFault.layer, nextEvtTime, nextEvtTime, 
                         elements.elements[currFault.layer][elementIndex], None)  
            queue.insertEvt(newFault)
            faultsTotal = faultsTotal + 1
            #newFault.infoOut()
    elif currFault.evtTime < currFault.initTime:
        print ("currFault.evtTime < currFault.initTime: Wrong evtTime!")
    
    if option == 1 or option == 2 or option == 3:
        #Impacted fault generation
        #generate impacted fault from the curFault based on the impact vector        
        if (currFault.impactVector != None) and (currFault.impactVector[2] >= THRESHOLD_IMPACTVECTOR):
            nextEvtTime = currFault.evtTime + DELTA
        
            if currFault.impactVector[0] > 0: #Topdown: if i is not the lowest layer, the impact vector will impact to i-1 layer 
                newFault = Fault(faultsTotal, currFault.initLayer, currFault.impactVector[0], currFault.initTime, nextEvtTime, 
                                 elements.elements[currFault.impactVector[0]][currFault.impactVector[1]], 
                                 generate_impactVector(currFault.impactVector[0]-1, N[currFault.impactVector[0]-1]))   
            else: #for the lowest layer, the impact vector is None
                newFault = Fault(faultsTotal, currFault.initLayer, currFault.impactVector[0], currFault.initTime, nextEvtTime, 
                         elements.elements[currFault.impactVector[0]][currFault.impactVector[1]], None)         
            queue.insertEvt(newFault)
            faultsTotal = faultsTotal + 1
            impactedFaultsTotal = impactedFaultsTotal + 1
            
    #Impact fault analysis and generation
    if option == 4:
        if (currFault.impactVector != None) and (currFault.impactVector[2] >= THRESHOLD_IMPACTVECTOR):
            nextEvtTime = currFault.evtTime + DELTA
        
            if currFault.impactVector[0] > 0: #Topdown: if i is not the lowest layer, the impact vector will impact to i-1 layer 
                newFault = Fault(faultsTotal, currFault.initLayer, currFault.impactVector[0], currFault.initTime, nextEvtTime, 
                                 elements.elements[currFault.impactVector[0]][currFault.impactVector[1]], 
                                 generate_impactVector(currFault.impactVector[0]-1, N[currFault.impactVector[0]-1]))   
            else: #for the lowest layer, the impact vector is None
                newFault = Fault(faultsTotal, currFault.initLayer, currFault.impactVector[0], currFault.initTime, nextEvtTime, 
                         elements.elements[currFault.impactVector[0]][currFault.impactVector[1]], None)  
             
            accuracy_correlation = random.random()
            accuracy_rca = random.random()
            if (accuracy_correlation < accuracyCorrelation): 
                if (accuracy_rca < accuracyRCA):
                    delayTotal = delayTotal + D_3
                    C_R = C_R + 1
                else: 
                    delayTotal = delayTotal + D_2
                    C_D = C_D + 1
            else: #insert only if the corellation analysis fails
                queue.insertEvt(newFault)
                faultsTotal = faultsTotal + 1
                impactedFaultsTotal = impactedFaultsTotal + 1

    """ 
    Monitoring and process the currFault
    """        
    #add code here to process the currFault
    if option == 1:
        #Calculate counters
        if currFault.assocElement.monitored == True:
            delayTotal = delayTotal + D_1
            C_M = C_M + 1
        else:
            delayTotal = delayTotal + D_0
        #currFault.infoOut()
    elif option == 2 or option == 3:
        #Calculate counters
        if currFault.assocElement.monitored == True:
            accuracy_fault_detection = random.random()
            accuracy_rca = random.random()
            if (accuracy_fault_detection < accuracyFaultDetection): 
                if (accuracy_rca < accuracyRCA):
                    delayTotal  = delayTotal + D_3
                    C_R = C_R + 1
                else:   
                    delayTotal = delayTotal + D_2
                    C_D = C_D + 1
            else:
                delayTotal = delayTotal + D_1
                C_M = C_M + 1
        else:
            delayTotal = delayTotal + D_0
        #currFault.infoOut()
    
    #update monitoring policy
    if option == 3 or option == 4:
        if faultsTotal > timesThresholdFaults*THRESHOLD_FAULTS:
            timesThresholdFaults = timesThresholdFaults + 1
            print("C_R/faultsTotal = " + str(C_R/faultsTotal) + 
                  ", THRESHOLD_UP = " + str(THRESHOLD_UP) + 
                  ", THRESHOLD_DOWN = " + str(THRESHOLD_DOWN))
            if C_R/faultsTotal > THRESHOLD_UP: #double check, different from the sudocode
                #too high, can decrease M_N and accuracy
                #update accuracy for detection and rca
                #decrease M_N
                newM_N = max(0, M_N * (1 - timesThresholdFaults*M_N_Change_Rate))
                elements.update_elements(newM_N)
                print("decrease to " + str(newM_N))
                elements.infoOut()
            elif C_R/faultsTotal < THRESHOLD_DOWN: 
                #too low, need to increase M_N and accuracy
                #update accuracy for detection and rca
                #increase M_N
                newM_N = min (1, M_N * (1 + timesThresholdFaults*M_N_Change_Rate))
                elements.update_elements(newM_N)
                print("increase to " + str(newM_N))
                elements.infoOut()
            
        
#Performance calculation    
print()
print("option = " + str(option))
print("Total num of faults: " + str(faultsTotal) + ", total num of impacted faults = " + str(impactedFaultsTotal))
print("C_M = " + str(C_M) + ", the ratio of monitored, but not detected faults = " + str(C_M/faultsTotal))
print("C_D = " + str(C_D) + ", the ratio of detected faults, but not diagnosed = " + str(C_D/faultsTotal))
print("C_R = " + str(C_R) + ", the ratio of diagnosed faults = " + str(C_R/faultsTotal))
print("FailureRate = " + str(failureRatio) + ", M/N = " + str(M_N) + ", FaultRecoverDelay = " + str(delayTotal/faultsTotal))



False True True False True False False True False True 

True True False True False False True True False False 

True False False False True False True True False True 

True True False True False True False False False True 

True False False True False False True True True False 

C_R/faultsTotal = 0.25374625374625376, THRESHOLD_UP = 0.9, THRESHOLD_DOWN = 0.4
increase to 0.8
True True True True True True True False True False 

True True True True True True False True False True 

False True True True True True True True False True 

False True True True True True True False True True 

False True True True True False True True True True 


option = 4
Total num of faults: 1760, total num of impacted faults = 254
C_M = 0, the ratio of monitored, but not detected faults = 0.0
C_D = 117, the ratio of detected faults, but not diagnosed = 0.06647727272727273
C_R = 462, the ratio of diagnosed faults = 0.2625
FailureRate = 0.3, M/N = 0.5, FaultRecoverDelay = 1.1198863636363636
